In [4]:
from datetime import datetime, timedelta
from pytz import timezone
import pytz

class TimeZone:
    FMT = '%Y-%m-%d %H:%M:%S %Z'

    def __init__(self, timezone_name='UTC'):
        self.timezone = timezone(timezone_name)

    def now(self):
        """Get current time in this timezone"""
        return datetime.now(self.timezone)
    
    def to_utc(self, dt):
        """Convert datetime to UTC"""
        if dt.tzinfo is None:
            dt = self.timezone.localize(dt)
        return dt.astimezone(pytz.UTC)
    
    def from_utc(self, utc_dt):
        """Convert UTC datetime to this timezone"""
        if utc_dt.tzinfo is None:
            utc_dt = pytz.UTC.localize(utc_dt)
        return utc_dt.astimezone(self.timezone)
    
    def format_datetime(self, dt):
        """Format datetime using TimeZone.FMT"""
        return dt.strftime(self.FMT)

time = TimeZone('US/Eastern')

loc = time.timezone.localize(datetime(2002, 10, 27, 6, 0, 0))
print(loc.strftime(TimeZone.FMT))

2002-10-27 06:00:00 EST


In [ ]:
class ConfirmationNumber:
    def __init__(self, account_number, transaction_code, time, time_utc, transaction_id):
        self.__account_number = account_number
        self.__transaction_code = transaction_code
        self.__time = time
        self.__time_utc = time_utc
        self.__transaction_id = transaction_id

    def __repr__(self):
        return f'ConfirmationNumber(account={self.__account_number}, transaction_code={self.__transaction_code}, utc_time={self.__time_utc}, transaction_id={self.__transaction_id})'

    @property
    def account_number(self):
        return self.__account_number

    @property
    def transaction_code(self):
        return self.__transaction_code
    
    @property
    def time(self):
        return self.__time

    @property
    def time_utc(self):
        return self.__time_utc

    @property
    def transaction_id(self):
        return self.__transaction_id

class BankAccount:
    active_accounts = set()
    interest_rate = 0.05
    transaction_number = 0

    def __init__(self, account_id, first_name, last_name, timezone_offset='UTC', balance=0):
        self.first_name = first_name
        self.last_name = last_name
        self._timezone = TimeZone(timezone_offset)
        self._balance = self.__validate_balance(balance)
        self.account_id = account_id

    @property
    def balance(self):
        return self._balance

    @property
    def account_id(self):
        return self._account_id
    
    @property
    def timezone_offset(self):
        return self._timezone.timezone.zone

    @property
    def full_name(self):
        return self.first_name + ' ' + self.last_name

    @timezone_offset.setter
    def timezone_offset(self, tz):
        if not isinstance(tz, str):
            raise TypeError('Timezone must be a str type')
        try:
            self._timezone = TimeZone(tz)  # Create new TimeZone instance
        except pytz.UnknownTimeZoneError:
            raise ValueError(f'Invalid timezone: {tz}. Must be a valid pytz timezone.')

    @account_id.setter
    def account_id(self, acc_id):
            if isinstance(acc_id, int):
                acc_id = str(acc_id)
            else:
                raise TypeError('Account number should be an int type')
            if acc_id not in BankAccount.active_accounts:
                BankAccount.active_accounts.add(acc_id)
                self._account_id = acc_id
            else:
                raise ValueError('This account is already existing')
            
    def __get_local_time(self):
        """Get current time in account's timezone"""
        return self._timezone.now()

    def __get_utc_time(self):
        """Get current UTC time"""
        return self._timezone.to_utc(self._timezone.now())

    def __validate_balance(self, balance):
        if not isinstance(balance, int):
            raise TypeError('Balance should be an int type')
        if balance < 0:
            raise ValueError('Balance cannot be negative')

        return balance

    def deposit(self, amount):
        try:
            self._balance += abs(amount)
        except TypeError:
            print('Failed to make a deposit. Please pass a valid number')
            return self.__generate_confirmation_number('X')
        return self.__generate_confirmation_number('D')

    def withdraw(self, amount):
        balance_after = self._balance - amount
        if balance_after < 0:
            print('You have not enough money on your balance')
            return self.__generate_confirmation_number('X')
        self._balance = balance_after
        return self.__generate_confirmation_number('W')

    def monthly_interest_rate(self):
        self._balance += self._balance * BankAccount.interest_rate
        return self.__generate_confirmation_number('I')

    def __generate_confirmation_number(self, code):

        current_time_local = datetime.now(self._timezone.timezone)
        current_time_utc = current_time_local.astimezone(pytz.UTC)

        BankAccount.transaction_number += 1
        confirmation = f"{code}-{self.account_id}-{current_time_utc.strftime('%Y%m%d%H%M%S')}-{BankAccount.transaction_number}"

        return confirmation

    @staticmethod
    def parse_confirmation_number(confirmation_number, preferred_timezone):
        splitted = confirmation_number.split('-')
        time_utc = datetime.strptime(splitted[2], '%Y%m%d%H%M%S')
        format_utc = time_utc.strftime('%Y-%m-%dT%H:%M:%S')
        
        # Use TimeZone class for timezone conversion
        preferred_tz_obj = TimeZone(preferred_timezone)
        utc_time_aware = pytz.UTC.localize(time_utc)
        local_time = utc_time_aware.astimezone(preferred_tz_obj.timezone)

        # Use TimeZone.FMT for consistent formatting
        format_local = local_time.strftime(TimeZone.FMT)

        result = {'transaction_code': splitted[0], 
         'account_number': splitted[1], 
         'time_utc': format_utc,
         'time': format_local, 
         'transaction_id': splitted[3]}

        return ConfirmationNumber(**result)

In [21]:
acc = BankAccount(123, 'Roma', 'Poma')

In [22]:
acc.account_id

'123'

In [23]:
acc = BankAccount(1234, 'Roma', 'Poma', timezone_offset='MST', balance=0)

In [24]:
acc.account_id

'1234'

In [25]:
BankAccount.active_accounts

{'123', '1234'}

In [26]:
acc.deposit(10)

'D-1234-20251014072109-1'

In [27]:
acc.balance

10

In [28]:
acc.withdraw(20)

You have not enough money on your balance


'X-1234-20251014072110-2'

In [29]:
acc.balance

10

In [30]:
acc.monthly_interest_rate()

'I-1234-20251014072111-3'

In [31]:
acc.balance

10.5

In [37]:
num = acc.parse_confirmation_number('I-1234-20251014070520-3', 'MST')

In [38]:
num.transaction_id

'3'

In [39]:
acc.timezone_offset

'MST'

In [40]:
num

ConfirmationNumber(account=1234, transaction_code=I, utc_time=2025-10-14T07:05:20, transaction_id=3)

In [46]:
import unittest

In [54]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [57]:
class TestBankAccount(unittest.TestCase):
    def create_account(self):
        acc = BankAccount(123, 'Rom', 'Pom', 'MST', -5)
        self.assertEqual(1, 1)

In [58]:
run_tests(TestBankAccount)


----------------------------------------------------------------------
Ran 0 tests in 0.000s

NO TESTS RAN
